# Text Classification Exam

Welcome to the Text Classification Practical Exam. In this exam, you will be tasked with building, training, and evaluating an NLP model to classify text data. You are provided with a labeled dataset containing both the text and its corresponding class labels.

Your objective is to develop a model that accurately predicts the class of the given text. Make sure to follow best practices in data preprocessing, model selection, and evaluation to achieve optimal results.

Good luck!
___

# Install and Import Needed Libraries

In [35]:
import pandas as pd

# Download the Dataset

In [36]:
!kaggle datasets download -d khaledzsa/spam-messages
!unzip spam-messages.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/spam-messages
License(s): unknown
spam-messages.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  spam-messages.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [37]:
df = pd.read_csv("/content/spam.csv")
df.head(10)

,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
5,FreeMsg Hey there darling it's been 3 week's n...,spam
6,Even my brother is not like to speak with me. ...,ham
7,As per your request 'Melle Melle (Oru Minnamin...,ham
8,WINNER!! As a valued network customer you have...,spam
9,Had your mobile 11 months or more? U R entitle...,spam


# Data Exploration

Before diving into preprocessing and model building, it’s important to first explore the dataset to understand its structure, distribution, and key characteristics. This step will help you gain insights into the data and guide your decisions in subsequent steps. Here’s what to consider:

1. **Inspect the Data**:
   Start by looking at the first few rows of the dataset to get a sense of its structure. Check the columns, data types, and a few sample entries. This helps to ensure that the data is loaded correctly and gives you an initial overview of the content.

2. **Check for Missing Values**:
   Identify if there are any missing values in the dataset.

3. **Distribution of Labels**:
   Examine the distribution of the target labels (classes).

4. **Text Data Characteristics (Bonus)**:
   Analyze the length of the text data. It is useful to calculate the number of words or characters in each text sample to understand how long the texts are. This will help you set a suitable `max_length` for tokenization and padding later. You can plot a histogram of text lengths to visualize the distribution.

5. **Common Words and Vocabulary (Bonus)**:
   Explore the most frequent words in the text data.

In [59]:
df.tail()

,text,label
5567,This 2nd time tried 2 contact u U å£750 Pound...,spam
5568,Will Ì b going esplanade fr home,ham
5569,"Pity, mood that So any suggestions",ham
5570,The guy bitching I acted like i'd interested b...,ham
5571,Rofl Its true name,ham


In [60]:
df.sample(3)

,text,label
1428,"Tell what, make little spreadsheet track whose...",ham
4269,Great Hope using connections mode men also co...,ham
3743,Nobody names penis girls name story add,ham


In [61]:
df.columns

Index(['text', 'label'], dtype='object')

In [41]:
df.isna().sum()

,0
text,0
label,0


In [42]:
df["label"].value_counts()

,count
label,
ham,4825
spam,747


# Data Cleaning and Preprocessing

In this section, we will focus on cleaning and filtering the dataset, preparing it for the text classification task. We will implement the following steps:

1. **Remove missing values**:  
   First, we eliminate any rows with missing values to ensure the dataset is complete and consistent.

2. **Filter by text length (Bonus)**:  
   To maintain a uniform dataset, we will filter the text samples by a specified word count range. This ensures that the texts are neither too short to lack context nor too long to introduce unnecessary complexity.

3. **English stopwords loading**:  
   We load a list of English stopwords to filter out commonly used but contextually insignificant words. This is an important step for improving the performance of the model, as stopwords do not contribute valuable information.

4. **Text cleaning**:  
   We apply a series of text cleaning steps to standardize and simplify the text data. This involves:
   
   - **Removing links (URLs)**:  
     Any URLs present in the text are removed as they are not meaningful for classification purposes.
   
   - **Removing special characters and punctuation**:  
     This step removes any non-alphabetical characters, ensuring the text only contains meaningful words.
   
   - **Lowercasing**:  
     All text is converted to lowercase for uniformity and to avoid case sensitivity issues.
   
   - **Removing English stopwords**:  
     Words that are part of the stopwords list are removed, as they do not add value to the classification task.
   
   - **Stemming or Lemmatization**:  
     Either stemming or lemmatization is applied to reduce words to their root or base form, ensuring consistency in word forms.

5. **Final cleanup**:
   Apply the cleanup function to the feature column.

By following these steps, the text will be cleaned, filtered, and ready for tokenization!

In [62]:
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import nltk
nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

def remove_stopwords_and_stem(text):
    words = text.split()
    words1 = [word for word in words if word not in stopwords]
    stem1 = ISRIStemmer()
    stem_words = [stem1.stem(word) for word in words1]
    return " ".join(stem_words)

df['text'] = df['text'].apply(remove_stopwords_and_stem)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
df['text'].head(10)

,text
0,"Go jurong point, crazy Available bugis n great..."
1,Ok lar Joking wif u oni
2,Free entry 2 wkly comp win FA Cup final tkts 2...
3,U dun say early hor U c already say
4,"Nah I think goes usf, lives around though"
5,FreeMsg Hey darling 3 week's word back I'd lik...
6,Even brother like speak They treat like aids p...
7,As per request 'Melle Melle Oru Minnaminungint...
8,WINNER As valued network customer selected rec...
9,Had mobile 11 months U R entitled Update lates...


In [45]:
import re

def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)
    return text

df['text'] = df['text'].apply(clean_text)


In [46]:
df.head(10)

,text,label
0,"Go jurong point, crazy.. Available bugis n gre...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry 2 wkly comp win FA Cup final tkts 2...,spam
3,U dun say early hor... U c already say...,ham
4,"Nah I think goes usf, lives around though",ham
5,FreeMsg Hey darling 3 week's word back! I'd li...,spam
6,Even brother like speak me. They treat like ai...,ham
7,As per request 'Melle Melle (Oru Minnaminungin...,ham
8,WINNER!! As valued network customer selected r...,spam
9,Had mobile 11 months more? U R entitled Update...,spam


In [47]:

def remove_punc(text):
  punc = '!@#$%^&*()_-+=}{~?/><|\}.'
  for x in text:
    if x in punc:
      text = text.replace(x, " ")
  return text

df['text'] = df['text'].apply(remove_punc)


In [48]:
df.head(10)

,text,label
0,"Go jurong point, crazy Available bugis n gre...",ham
1,Ok lar Joking wif u oni,ham
2,Free entry 2 wkly comp win FA Cup final tkts 2...,spam
3,U dun say early hor U c already say,ham
4,"Nah I think goes usf, lives around though",ham
5,FreeMsg Hey darling 3 week's word back I'd li...,spam
6,Even brother like speak me They treat like ai...,ham
7,As per request 'Melle Melle Oru Minnaminungin...,ham
8,WINNER As valued network customer selected r...,spam
9,Had mobile 11 months more U R entitled Update...,spam


# Tokenization, Padding, and Data Splitting

In this step, we will prepare the text data for input into a model by converting the text into numerical sequences, padding them to a uniform length, and splitting the dataset into training and testing sets. Here's an overview of the steps involved:

1. **Tokenization**:
   We use a tokenizer to convert the cleaned text into numerical sequences. You can use `Tokenizer` tokenizer from `tensorflow.keras.preprocessing.text` package or any other tokenizer you like.

2. **Text to sequences**:
   After fitting the tokenizer on the cleaned text, we transform each text into a sequence of numbers, where each number corresponds to a token (word) in the text.

3. **Padding the sequences**:
   Since different texts may vary in length, we pad the sequences to ensure they all have the same length.

4. **Label encoding**:
   The labels (target values) also need to be converted into numerical form if they are not encoded.

5. **Train-test split**:
   The dataset is divided into training and testing sets. We allocate 80% of the data for training the model and reserve 20% for testing its performance.
   
   - The **training data** consists of the padded sequences used to train the model.
   - The **training labels** are the encoded labels corresponding to the training data.
   - The **testing data** is used to assess the model’s performance after training.
   - The **testing labels** are the encoded labels corresponding to the testing data.

6. **Data shape confirmation**:
   After splitting the data, we print the shape (dimensions) of both the training and testing sets to confirm that the data is properly divided and formatted.

By the end of this step, the text data will be transformed into padded numerical sequences, the labels will be encoded, and the data will be split into training and testing sets for model development and evaluation.

In [49]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


token = Tokenizer()
token.fit_on_texts(df['text'])
sequences = token.texts_to_sequences(df['text'])

Voc_size= len(token.word_index) + 1

In [50]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 500
pad_seq = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

label_encoder = LabelEncoder()
label = label_encoder.fit_transform(df['label'])

In [51]:
X_train, X_test, y_train, y_test = train_test_split(pad_seq, label, test_size=0.2, random_state=42)

In [52]:
print("X_train : ",X_train.shape)
print("X_test : ", X_test.shape)
print("y_train : ",y_train.shape)
print("y_test",y_test.shape)


X_train :  (4457, 500)
X_test :  (1115, 500)
y_train :  (4457,)
y_test (1115,)


# Building the Classifier

In this step, you will design and build a NLP Classifier model to classify text data. Below is a breakdown of the key components you'll implement, but it's up to you to decide how to configure them based on your understanding and experimentation:

1. **Model Type**:
   You will use a Sequential model, which allows you to stack layers in a linear sequence.

2. **Input Layer**:
   Define the shape of the input data. Consider the dimensions of your padded sequences and set the input shape accordingly.

3. **Embedding Layer**:
   The embedding layer will convert input tokens (integers) into dense vector representations. You will need to determine the size of the input dimension (based on your vocabulary) and the output dimension (embedding size).

4. **Bidirectional Simple RNN/LSTM Layers**:
   You can add one or more recurrent layers. Consider using Bidirectional layers to capture contextual information from both directions (forward and backward). You can chose SimpleRNN/GRU/LSTM to perform this step.

5. **Dense Layers**:
   Add one or more fully connected (Dense) layers to process the output from the RNN/GRU/LSTM layers.

6. **Output Layer**:
   The output layer should match the type of classification task you're working on. Consider using appropriate activation function with appropriate number of units.

7. **Model Summary**:
   After defining your model architecture, print a summary to review the number of layers, types of layers, and total parameters.

8. **Model Compilation**:
   Finally, compile the model by selecting an optimizer, a loss function, and metrics.

In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN,Dropout,Embedding,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [81]:
model = Sequential([

    Embedding(input_dim=Voc_size, output_dim=128, input_length=max_length),
    Bidirectional(SimpleRNN(128, return_sequences=True)),
    Bidirectional(SimpleRNN(64)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

Early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_7 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Defining Batch Size, Creating Datasets, and Training the Model

In this step, you will define the batch size, create TensorFlow Datasets for both training and testing, and train the model. The key elements to consider are outlined below, and it is up to you to choose the specific configurations based on your preferences and experimentation:

1. **Batch Size**:
   Select a batch size for training and testing. The batch size determines how many samples will be processed together in one forward and backward pass during training.

2. **Creating Datasets**:
   Use TensorFlow’s `Dataset.from_tensor_slices()` to create datasets from the training and testing data.

3. **Batching the Datasets**:
   Batch the datasets by grouping the data into batches of the specified size.

4. **Training the Model**:
   Train the model by fitting it on the training dataset for a specified number of epochs. You will also need to provide the validation data to monitor the model’s performance on unseen data during training.

5. **Tracking Training History**:
   During training, the model’s performance metrics (such as loss and accuracy) will be tracked over the epochs, and the results will be stored in the `history` object.

In [79]:

batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)


In [56]:
history = model.fit(train_dataset,epochs=10, callbacks=Early_stopping)

Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 220ms/step - accuracy: 0.8852 - loss: 0.2895
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


70/70 ━━━━━━━━━━━━━━━━━━━━ 17s 202ms/step - accuracy: 0.9853 - loss: 0.0557
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 207ms/step - accuracy: 0.9965 - loss: 0.0099
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 203ms/step - accuracy: 0.9983 - loss: 0.0063
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 206ms/step - accuracy: 0.9978 - loss: 0.0093
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 193ms/step - accuracy: 0.9981 - loss: 0.0044
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 194ms/step - accuracy: 0.9995 - loss: 0.0032
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 13s 192ms/step - accuracy: 1.0000 - loss: 1.8898e-04
Epoch 9/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 193ms/step - accuracy: 1.0000 - loss: 1.1023e-04
Epoch 10/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 17s 244ms/step - accuracy: 1.0000 - loss: 8.7465e-05


# Model Evaluation

Once the model is trained, the next step is to evaluate its performance on the testing dataset.

1. **Evaluate the Model**:
   You will use the `evaluate()` method to assess the model’s performance on the test dataset.

2. **Testing Dataset**:
   Ensure that the testing dataset is properly prepared and batched, just like the training dataset.

4. **Loss Curve**:
   A loss curve plots the loss values for both the training and validation datasets over the epochs.

In [57]:
model.evaluate(test_dataset)

18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.9881 - loss: 0.0770


[0.10603067278862, 0.9856502413749695]

# Model Inference

In this step, you will use the trained model to make predictions on new, unseen data (inference). Here’s an outline of the key points:

1. **Create Test Sample**:
   Create a string to test your modelm the goal here is to give the model Before making predictions, ensure that the new data is preprocessed in the same way as the training data. This includes tokenization, padding, and any other transformations you applied during the data preprocessing step. The data can be single text to see the result of the prediction.

2. **Model Prediction**:
   Use the `predict()` method to feed new samples into the trained model and obtain predictions. The model will output probabilities or predicted class labels based on the type of classification task (binary or multi-class).

3. **Interpreting Predictions**:
   The model will return probabilities for each class.

In [76]:

sample = "I want to get the hightest marks in this exam doctor ali ^_^"

sample = clean_text(sample)
sample = remove_punc(sample)
sample = remove_stopwords_and_stem(sample)
pos_sequence = token.texts_to_sequences([sample])
pos_padded_sequence = pad_sequences(pos_sequence, maxlen=max_length, padding='post', truncating='post')

print(sample)

pred = model.predict(pos_padded_sequence)

I want get hightest marks exam doctor ali
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


# Notebook Question:
- How did you handle text preprocessing? Why did you choose this approach?

I used a muliable functions to reomve URls and punc and stopwords and i did stemming.

- Why did you choose this model design?

i chosed this model design because it best sutied me for the accuracy.

- Why did you pick this number of layers or units for the model?
because it sutied my model and i didn't have overfitting or underfitting.


- Why did you select these evaluation methods?

i used them to show how my model preform.

- Does your model show signs of overfitting or underfitting? How do you know?

as i sea it, there is no overfitting or underfitting

- What changes could you make to improve the model and fix overfitting or underfitting?


remove some layers, or do a dropout.

Answer Here: